In [1]:
import clip
import argparse
import numpy as np

from loaders import CxrDataLoader
from classifier.model import ChexpertClassifier
from classifier.trainer import ChexpertClassifierTrainer

In [2]:
LABELS = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
          'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
          'Support Devices']

args = argparse.Namespace(
    dataset_name='iu-xray',
    batch_size=16,
    num_workers=0,
    max_seq_length=248
)

In [3]:
clip_model, preprocess = clip.load("ViT-B/32", load_from_clip=True)
clip_model.eval()

vision_model = clip_model.visual
input_dim = clip_model.vision_width
hidden_dim = clip_model.transformer_width

model = ChexpertClassifier(vision_model, input_dim, hidden_dim, num_classes=len(LABELS))

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")

Model parameters: 88,250,126


In [4]:
dataloaders = {
    'train': CxrDataLoader(args, split='train', transform=preprocess),
    'val': CxrDataLoader(args, split='val', transform=preprocess),
    'test': CxrDataLoader(args, split='test', transform=preprocess)
}

In [5]:
trainer = ChexpertClassifierTrainer(model, train_loader=dataloaders['train'], val_loader=dataloaders['val'], log_interval=10)

03/19/2025 01:04:31 - INFO - Total parameters: 88,250,126
03/19/2025 01:04:31 - INFO - Trainable parameters: 400,910 (0.45%)


In [6]:
# TODO: check if evaluation metrics are computed correctly: log predicted labels
# TODO: handle uncertain labels
trainer.train(epochs=1)

03/19/2025 01:04:31 - INFO - Start training
03/19/2025 01:04:31 - INFO - Starting epoch 1/1
100%|██████████| 169/169 [02:20<00:00,  1.20it/s]
03/19/2025 01:06:52 - INFO - Epoch 1 completed in 140.40s. Average loss: 0.6961
03/19/2025 01:06:52 - INFO - Running evaluation on val set
Val Evaluation: 100%|██████████| 37/37 [00:30<00:00,  1.23it/s]
03/19/2025 01:07:22 - INFO - Val metrics: {
  "val_loss": 0.6931474804878235,
  "val_auc": 0.5307488956400571,
  "val_precision": 0.0,
  "val_recall": 0.0,
  "val_f1": 0.0
}
03/19/2025 01:07:22 - INFO - Training finished
